In [57]:
import pathlib
import pandas as pd
import re
from collections import Counter
from fuzzywuzzy import fuzz

## Read parquets with data

In [54]:
df_company = pd.read_parquet(pathlib.Path("data/company_info.parquet"))
df_utes = pd.read_parquet(pathlib.Path("data/utes.parquet"))

In [56]:
# Create a new dataframe with all the company names that are not in the utes
df_not_in_utes = df_company[~df_company['FullName'].isin(df_utes['FullName'])]
df_not_in_utes

,NIF,FullName,Name,Province,NIFtype,CompanyType,CompanyDescription,id_tender
0,00021492x,carmen balguerias jiménez,carmen balguerias jiménez,None,DNI,None,None,[https://contrataciondelestado.es/sindicacion/...
1,00035211k,paloma sáinz de la maza de la serna,paloma sáinz de la maza de la serna,None,DNI,None,None,[https://contrataciondelestado.es/sindicacion/...
2,00067665e,alberto delgado cebrián,alberto delgado cebrián,None,DNI,None,None,[https://contrataciondelestado.es/sindicacion/...
3,00072839k,fernandez abad vicente,fernandez abad vicente,None,DNI,None,None,[https://contrataciondelestado.es/sindicacion/...
4,00076938a,luis terán lópez,luis terán lópez,None,DNI,None,None,[https://contrataciondelestado.es/sindicacion/...
...,...,...,...,...,...,...,...,...
267238,b28672764,xylem water solutions españa s.l.u.,xylem water solutions españa,Madrid,CIF,s.l.,Sociedades de responsabilidad limitada,[https://contrataciondelestado.es/sindicacion/...
267239,b99289795,zaragoza y eventos s.l.,zaragoza y eventos,Zaragoza,CIF,s.l.,Sociedades de responsabilidad limitada,[https://contrataciondelestado.es/sindicacion/...
267240,a28011153,zardoya otis s.a.,zardoya otis,Madrid,CIF,s.a.,Sociedades anónimas,[https://contrataciondelestado.es/sindicacion/...
267241,b45273406,zenith toledo,zenith toledo,Toledo,CIF,s.l.,Sociedades de responsabilidad limitada,[https://contrataciondelestado.es/sindicacion/...


## Get utes names split

### Auxiliary functions

In [10]:
def remove_substring(string, substring):
    """
    Remove substring from string

    Parameters
    ----------
    string : str
        String to be processed
    substring : str
        Substring to be removed from string

    Returns
    -------
    result : str
        String without the substring
    """
    pattern = re.escape(substring)
    result = re.sub(pattern, " ", string)
    return result


def eliminate_patterns(text):
    """
    Eliminate fixed patterns found in several utes (e.g., "ley" + month, "u.t.e." prefix/suffix, etc.)

    Parameters
    ----------
    text : str
        String to be processed

    Returns
    -------
    result : str
        String without the fixed patterns
    """

    # Patterns to be removed from the inut string
    PATTERNS = [
        r"ley.*?(enero|febrero|marzo|abril|mayo|junio|julio|agosto|septiembre|octubre|noviembre|diciembre)",
        r"revista.*?\d+",
        r"expte.*?\d+",
        r"ley.*\d+",
        r"contr.*\d+"
    ]
    combined_pattern = re.compile("|".join(PATTERNS), flags=re.IGNORECASE)

    # Remove patterns
    result = re.sub(combined_pattern, "", text)

    # Remove additional strings
    EXP = ["u.t.e.", "u.t.e", "union temporal de empresas",
           "compromiso de ", "abreviadamente", "compromiso"]
    for substring in EXP:
        result = remove_substring(result, substring)

    return result.strip()


def remove_first_substring(text, substring):
    """Remove only the first occurrence of a substring from a text.

    Parameters
    ----------
    text : str
        Text to be processed
    substring : str
        Substring to be removed from text

    Returns
    -------
    result : str
        Text without the substring
    """

    index = text.find(substring)
    if index != -1:
        return text[:index] + text[index + len(substring):]
    return text


def extract_difference(str1, str2):
    """Extract the difference between two strings.

    Parameters
    ----------
    str1 : str
        String to be processed
    str2 : str
        String to be processed

    Returns
    -------
    result : str
        Difference between str1 and str2
    """

    names1 = str1.split()
    names2 = set(str2.split())
    difference = sorted(set(names1) - names2, key=lambda x: names1.index(x))
    result = ' '.join(difference)

    return result


def get_splits_additional_rules(ute_rem, split_rules):
    """Splits a given string (ute_rem) based on set of split_rules and returns a list of split names.

    Parameters
    ----------
    ute_rem : str
        The input string to be split using additional split rules.
    split_rules : list
        A list of split rules to be applied on the input string.

    Returns
    -------
    split_names : list
        A list of strings resulting from splitting the input string based on the given rules.
    """

    # Filter out only the relevant split rules that exist in ute_rem
    split_rules = [rule for rule in split_rules if rule in ute_rem]

    if len(split_rules) == 1:
        # If only one rule is found, split ute_rem using that rule and strip any leading/trailing whitespaces
        split_names = [name.strip() for name in ute_rem.split(split_rules[0])]
    else:
        # If multiple split rules are found, sort them based on their first occurrence in the ute_rem
        split_rules.sort(key=lambda rule: ute_rem.index(
            rule) if rule in ute_rem else len(ute_rem))

        # Create a list of split names by applying each split rule on the remaining ute_rem and stripping leading whitespaces
        split_names = []
        for i, split_rule in enumerate(split_rules):
            found = ute_rem.split(split_rule)[0].strip()
            split_names.append(found)
            ute_rem = remove_substring(ute_rem, found)
            if i == len(split_rules)-1:
                split_names.append(ute_rem)

    return split_names


def split_names(row):

    # Remove fixed patterns found in several utes (e.g., "ley" + month, "u.t.e." prefix/suffix, etc.)
    ute_rem = eliminate_patterns(row)

    # Look for utes in the form "letter - number word" or "word letter - number". If so, we keep it as it is
    if re.search(r"([a-zA-Z]) - (\d+) (\S+)", ute_rem) or re.search(r"(\b\w+\b) ([a-zA-Z]) - (\d+)", ute_rem):
        return [ute_rem.strip()]

    # Possible split rules
    look_first_split_rules = \
        [
            "s.l.p.", "s.l.p", "s.l.l.", "s.a.u.", "s.l.u.",
            "s.l.u", "s.l.u,", "slu", "s.l", "c.o.o.p.", "s.a",
            "sl.", "sccl", "s.coop.pequeña"
        ]
    split_rules = [el for el in df_company.CompanyType.unique().tolist(
    ) if el != None and el != 'u.t.e.'] + look_first_split_rules
    additional_split_rules = ["-", "_", ",", "+"]

    # Sort according to size and scape characters in split_rules
    split_rules = sorted(split_rules, key=len, reverse=True)
    scaped_split_rules = [rule.replace('.', '\.').replace(
        ' ', '\s') for rule in split_rules]

    # Find rules
    found_rules = []
    aux = ute_rem
    for rule, escaped_rule in zip(split_rules, scaped_split_rules):
        if re.search(rf'{escaped_rule}', aux):
            occurrences = len(re.findall(rf'{escaped_rule}', aux))
            for _ in range(occurrences):
                found_rules.append(rule)
                aux = remove_first_substring(aux, rule)

    # Check if there are additional rules
    has_additional_rule = [
        True if rule in ute_rem else False for rule in additional_split_rules]
    
    # If there are found rules according to ute type
    if len(found_rules) > 0:

        # Remove additional_split_rules followed by an ute split_rule
        for element in split_rules:
            for subelement in additional_split_rules:
                pattern = f'{subelement}{element}'
                if pattern in ute_rem:
                    ute_rem = ute_rem.replace(pattern, f' {element}')

        # Check if there are still additional rules
        has_additional_rule = [
            True if rule in ute_rem else False for rule in additional_split_rules]

        # Order according to appearance in ute_rem
        found_rules.sort(key=lambda rule: ute_rem.index(
            rule) if rule in ute_rem else len(ute_rem))

        # If there is only one rule and it is at the end of the string
        if len(found_rules) == 1 and ute_rem.endswith(found_rules[0]):
            # If there are no additional rules
            if not any(has_additional_rule):
                # If the rule is "y" we split the string in two; otherwise, we don't split the string
                if "y" in ute_rem:
                    split_names = get_splits_additional_rules(ute_rem, ["y"])
                else:
                    split_names = [ute_rem.strip()]
            else:
                # If there are additional rules, we split the string according to them
                split_names = get_splits_additional_rules(
                    ute_rem, additional_split_rules)

        else:
            if not any(has_additional_rule):
                split_names = []
                for i, split_rule in enumerate(found_rules):
                    found = ute_rem.split(split_rule)[
                        0].strip() + " " + split_rule
                    split_names.append(found)
                    ute_rem = remove_substring(ute_rem, found)
                    if i == len(split_rules)-1:
                        split_names.append(ute_rem)
            else:
                new_split_names = []
                split_names = get_splits_additional_rules(
                    ute_rem, additional_split_rules)
                for el in split_names:
                    go = [True if rule in el else False for rule in found_rules]
                    if any(go):
                        for i, split_rule in enumerate(found_rules):
                            found = el.split(split_rule)[
                                0].strip() + " " + split_rule
                            new_split_names.append(found)
                            el = remove_substring(el, found)

                            if i == len(found_rules) - 1:
                                new_split_names.append(el)
                    else:
                        new_split_names.append(el)
                split_names = new_split_names

    elif True in has_additional_rule:
        split_names = get_splits_additional_rules(
            ute_rem, additional_split_rules + [" y "])

    elif re.compile(r'\s+y\s+').search(ute_rem):
        split_names = get_splits_additional_rules(ute_rem, [" y "])

    else:

        substrings = [" ".join(ute_rem.split()[i:j]) for i in range(
            len(ute_rem.split())) for j in range(i + 1, len(ute_rem.split()) + 1)]

        split_names = [
            el for el in substrings if el in df_not_in_utes.FullName.values.tolist()]

        if len(split_names) == 1:
            split_names += [extract_difference(ute_rem, split_names[0])]

        elif len(split_names) == 0:
            split_names = [ute_rem.strip()]

    # Cleaning errors after splitting
    split_names = [name.strip("-_ ,,+y").strip()
                   for name in split_names if len(name.strip()) > 1]

    return split_names

In [ ]:
df_utes['split_names'] = df_utes['FullName'].apply(split_names)
df_utes['utes_length'] = df_utes.split_names.apply(len)

In [42]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       'display.max_colwidth',None
                       ):
    display(df_utes[df_utes.utes_length<1][["FullName", "split_names"]])

,FullName,split_names
260127,u.t.e. ley 18 82 numero 1,[]


## Get companies' utes from splits

In [ ]:
def get_company_utes_from_splits(row, df_utes):
    
    thr = 95
    utes = [
        df_utes.FullName.values[el]
        for el in range(len(df_utes))
        for comp in df_utes.split_names.values[el]
        if fuzz.ratio(row.FullName, comp) > thr
    ]

    return utes  

In [ ]:
df_not_in_utes["utes"] = df_not_in_utes.apply(lambda row: get_company_utes_from_splits(row, df_utes), axis=1)
df_not_in_utes['utes_length'] = df_not_in_utes.utes.apply(len)
df_not_in_utes[(df_not_in_utes.utes_length>0)]

## Get companies' utes from raw

In [4]:
def encontrar_substring_similar(main_string, substring, threshold=95):
    
    for i in range(len(main_string) - len(substring) + 1):
        sub = main_string[i:i + len(substring)]
        if not re.search(r'\b' + re.escape(sub) + r'\b', main_string):
            continue

        similitud = fuzz.ratio(sub, substring)
        if similitud >= threshold:
            return True
    return False

def get_company_utes(row, df_utes):
    pattern = re.compile(r'\b' + re.escape(row['Name']) + r'\b', re.IGNORECASE)
    utes = [fullName for fullName in df_utes['FullName'] if encontrar_substring_similar(fullName, row['Name'])]
    return utes

In [ ]:
df_not_in_utes["utes"] = df_not_in_utes.apply(lambda row: get_company_utes(row, df_utes), axis=1)
df_not_in_utes['utes_length'] = df_not_in_utes.utes.apply(len)
df_not_in_utes[(df_not_in_utes.utes_length>0)]

In [ ]:
df_not_in_utes.to_parquet("data/df_not_in_utes_enriched.parquet")

### Sample

In [6]:
df_aux = df_not_in_utes.sample(frac=0.005, replace=True, random_state=1)
df_aux["utes"] = df_aux.apply(lambda row: get_company_utes(row, df_utes), axis=1)
df_aux['utes_length'] = df_aux.utes.apply(len)
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                      'display.max_colwidth',None
                       ):
    #display(df_aux[["FullName", "utes"]])
    display(df_aux[df_aux.utes_length > 1][["FullName", "utes"]])
print(len(df_aux[df_aux.utes_length < 1]) / len(df_aux))

,FullName,utes
132920,woover trends s.l.u.,"[u.t.e. vodafone españa s.a.u. everis spain s.l.u. woover trends s.l.u., u.t.e. deloitte consulting s.l.u. woover trends s.l.u.]"
145030,rotorsun s.l.,"[orthem servicios y actuaciones ambientales s.a.u. y rotorsun s.l., rotorsun s.l.y orthem s.a.u.ute, u.t.e. rotorsun,s.l.y ortem s.a.u.]"
178526,efs mantenimiento y servicios tecnicos s.l.,"[u.t.e. efs mantenimiento y servicios tecnicos s.l.u. - adelte transporte y servicios efs s.l., u.t.e. efs mantenimiento y servicios tecnicos s.l.u. eurofred s.a., u.t.e. efs mantenimiento y servicios técnicos s.l.u u.t.e. ley 18 82 26 - 05 u.t.e. efs adelte cond.mad]"
157991,"ingeniería,obras y tecnología europea s.l.","[u.t.e. ingenieria,obras y tecnologia europea s.l. jarypark s.l., u.t.e. ecocivil electromur g.e.s.l.e ingeniería,obras y tecnología europea s.l. abreviadamente u.t.e. vaguada .]"


0.9900763358778626


## Tests

In [51]:
df_aux = df_company[(df_company['FullName'].str.contains('orega s.l'))]# & (df_company['CompanyType'] != "u.t.e.")
df_aux = df_not_in_utes.sample(frac=0.005, replace=True, random_state=2)
df_aux["utes"] = df_aux.apply(lambda row: get_company_utes2(row, df_utes), axis=1)
df_aux['utes_length'] = df_aux.utes.apply(len)
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                      'display.max_colwidth',None
                       ):
    display(df_aux[df_aux.utes_length > 1][["FullName", "utes"]])
print(len(df_aux[df_aux.utes_length < 1]))

,FullName,utes
97014,construcciones sevilla nevado s.a.,"[u.t.e. construcciones sevilla nevado s.a. sebastián sevilla nevado s.l.u. josé carmona e hijos s.l., construcciones sevilla nevado s.a. cimasa empresa de construccion e ingenieria s.l. u.t.e]"
99542,indra sistemas s.a.,"[indra sistemas s.a.e. instalaciones, u.t.e. indra sistemas s.a.y alisea esco s.a u.t.e. jaen, kapsch trafficom transportation s.a. - indra sistgemas s.a. worldline iberia s.a u.t.e., u.t.e. telefonica de españa s.a.u. indra sistemas s.a., indra sistemas s.a. connectis consulting services,s.a.u.ute, u.t.e. indra sistemas s.a. - sistemas y montajes industriales s.a., u.t.e. indra sistemas s.a.y sotel it solutions s.l., indra sistemas s.a. aerum aviation group,s.l.ute, u.t.e. indra sistemas s.a.integracion tecnologica empresarial u.t.e. ley 18 1982 26 mayo 201853a1, u.t.e. alfatec sistemas s.l. - indra sistemas s.a.]"
106776,tecnologia de firmes s.a.,"[u.t.e. tecnologia de firmes s.a. - guerola transer s.l.u., u.t.e. tecnologia de firmes s.a. - asfaltecno obras y servicios s.a., u.t.e. tecnologia de firmes s.a. - construcciones y obras llorente s.a., tecnologia de firmes s.a. y eiffage infraestructuras s.a.u u.t.e., u.t.e. tecnologia de firmes s.a. - global de electricidad e instalaciones s.l. - artectum s.l., u.t.e. tecnologia de firmes s.a. y constructora consvial s.l u.t.e. ampliacion sala aeropuerto v, u.t.e. guerola transer s.l.u. - tecnologia de firmes s.a., u.t.e. tecnologia de firmes s.a. - gerola transfer s.l.u., u.t.e. tecnologias de firme s.a. arcadi ingenieria e instalaciones innovadoras s.l., u.t.e. tecnologa de firmes s.a. montajes electricos antonio godoy s.l., u.t.e. tecnologia de firmes s.a. eiffage infraestructuras s.a.u., asfaltos y const elsan s.a. tecnologias de firmes s.a.]"


1300


In [125]:
df_aux = df_company[(df_company['FullName'] == 'civit s.l.') | (df_company['FullName'] == 'construcciones sevilla nevado s.a.')]# & (df_company['CompanyType'] != "u.t.e.")
df_aux

,NIF,FullName,Name,Province,NIFtype,CompanyType,CompanyDescription,id_tender
97014,a10036424,construcciones sevilla nevado s.a.,construcciones sevilla nevado,Cáceres,CIF,s.a.,Sociedades anónimas,[https://contrataciondelestado.es/sindicacion/...
118563,b08452799,civit s.l.,civit,Barcelona,CIF,s.l.,Sociedades de responsabilidad limitada,[https://contrataciondelestado.es/sindicacion/...


In [ ]:
#df_aux = df_company[(df_company['FullName'].str.contains('tecnologia de firmes s.a.'))]# & (df_company['CompanyType'] != "u.t.e.")
df_aux = df_not_in_utes.sample(frac=0.005, replace=True, random_state=1)
df_aux["utes"] = df_aux.apply(lambda row: get_company_utes(row, df_utes), axis=1)
df_aux['utes_length'] = df_aux.utes.apply(len)
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                      'display.max_colwidth',None
                       ):
    display(df_aux[["FullName", "utes"]])
    #display(df_aux[df_aux.utes_length > 1][["FullName", "utes"]])
print(len(df_aux[df_aux.utes_length < 1]))

In [ ]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                      'display.max_colwidth',None
                       ):
    display(df_aux[df_aux.utes_length > 1][["FullName", "utes"]])

In [ ]:
# found before 1583
df_not_in_utes["utes"] = df_not_in_utes.apply(lambda row: get_company_utes2(row, df_utes), axis=1)
df_not_in_utes['utes_length'] = df_not_in_utes.utes.apply(len)
df_not_in_utes[(df_not_in_utes.utes_length>0)]

In [ ]:
df_aux2 = df_not_in_utes.copy()
df_aux2["utes"] = df_aux2.apply(lambda row: get_company_utes2(row, df_utes), axis=1)
df_aux2['utes_length'] = df_aux2.utes.apply(len)
df_aux2[(df_aux2.utes_length>0)]

In [ ]:
df_not_in_utes.to_parquet("data/df_not_in_utes_enriched.parquet")

## Check coverage

In [46]:
df_not_in_utes_enriched1 = pd.read_parquet(pathlib.Path("data/df_not_in_utes_enriched1.parquet"))
filtered = df_not_in_utes_enriched1[(df_not_in_utes_enriched1.utes_length>0)]
filtered

,NIF,FullName,Name,Province,NIFtype,CompanyType,CompanyDescription,id_tender,utes,utes_length
943,01939568r,alejandro,alejandro,None,DNI,None,None,[https://contrataciondelestado.es/sindicacion/...,[xoaquín monteagudo romero alejandro martín ló...,2
1229,02497147z,alicia torres gonzález,alicia torres gonzález,None,DNI,None,None,[https://contrataciondelestado.es/sindicacion/...,[u.t.e. alicia torres gonzález y artec 4 s.l.p.],1
8765,07270702b,manuel jesus piriz gil,manuel jesus piriz gil,None,DNI,None,None,[https://contrataciondelestado.es/sindicacion/...,[manuel jesus piriz gil - maria navarro cifuen...,1
12498,09331668q,joaquín,joaquín,None,DNI,None,None,[https://contrataciondelestado.es/sindicacion/...,[josep maria mezquida casases joaquín solé mir...,1
13477,10017354a,eloy santin castañeiras,eloy santin castañeiras,None,DNI,None,None,[https://contrataciondelestado.es/sindicacion/...,[alberto jose garcía martínez y eloy santín ca...,1
...,...,...,...,...,...,...,...,...,...,...
267192,a79524054,urbaser s.a.,urbaser,Madrid,CIF,s.a.,Sociedades anónimas,[https://contrataciondelestado.es/sindicacion/...,[u.t.e. biziss movilidad urbana sostenible s.l...,13
267215,a23434970,vialterra infraestructuras s.a.,vialterra infraestructuras,Jaén,CIF,s.a.,Sociedades anónimas,[https://contrataciondelestado.es/sindicacion/...,[u.t.e. vialterra infraestructuras s.a. kerkro...,4
267224,a80907397,vodafone españa s.a.u.,vodafone españa,Madrid,CIF,s.a.,Sociedades anónimas,[https://contrataciondelestado.es/sindicacion/...,[u.t.e. vodafone españa s.a.u. everis spain s....,6
267225,a62186556,vodafone ono s.a.u.,vodafone ono,Barcelona,CIF,s.a.,Sociedades anónimas,[https://contrataciondelestado.es/sindicacion/...,"[u.t.e. vodafone españa s.a.u. vodafone ono,s....",1


In [52]:
utes_found = [ute for el in filtered.utes.values.tolist() for ute in el]
utes_matching = (len(list(set(utes_found))) / len(df_utes)) * 100
print(utes_matching)

38.70905956713273


In [60]:
df_not_in_utes_enriched1 = pd.read_parquet(pathlib.Path("data/utes_spark.parquet"))
df_not_in_utes_enriched1["utes_length"] = df_not_in_utes_enriched1["utes"].apply(len)
filtered = df_not_in_utes_enriched1[(df_not_in_utes_enriched1.utes_length>0)]
filtered

,NIF,FullName,Name,Province,NIFtype,CompanyType,CompanyDescription,id_tender,utes,utes_length
637,01178902z,rosa granados,rosa granados,None,DNI,None,None,[https://contrataciondelestado.es/sindicacion/...,[gesnaer consulting s.l.n.e. rosa granados u.t...,1
943,01939568r,alejandro,alejandro,None,DNI,None,None,[https://contrataciondelestado.es/sindicacion/...,"[u.t.e. antonio lópez sánchez,alejandro martín...",5
1213,02459022t,andres perea ortega,andres perea ortega,None,DNI,None,None,[https://contrataciondelestado.es/sindicacion/...,[euroestudios s.l. andres perea ortega u.t.e.],1
1229,02497147z,alicia torres gonzález,alicia torres gonzález,None,DNI,None,None,[https://contrataciondelestado.es/sindicacion/...,[u.t.e. alicia torres gonzález y artec 4 s.l.p.],1
8493,07016148k,construcciones perez,construcciones perez,None,DNI,None,None,[https://contrataciondelestado.es/sindicacion/...,[u.t.e. construcciones perez jimenez s.l. alba...,2
...,...,...,...,...,...,...,...,...,...,...
261969,a79524054,urbaser s.a.,urbaser,Madrid,CIF,s.a.,Sociedades anónimas,[https://contrataciondelestado.es/sindicacion/...,[u.t.e. ambitec servicios ambientales s.a.u. y...,18
261992,a23434970,vialterra infraestructuras s.a.,vialterra infraestructuras,Jaén,CIF,s.a.,Sociedades anónimas,[https://contrataciondelestado.es/sindicacion/...,[u.t.e. vialterra infraestructuras s.a. desarr...,4
262001,a80907397,vodafone españa s.a.u.,vodafone españa,Madrid,CIF,s.a.,Sociedades anónimas,[https://contrataciondelestado.es/sindicacion/...,"[u.t.e. vodafone españa s.a.u. vodafone ono,s....",8
262002,a62186556,vodafone ono s.a.u.,vodafone ono,Barcelona,CIF,s.a.,Sociedades anónimas,[https://contrataciondelestado.es/sindicacion/...,"[u.t.e. vodafone españa s.a.u. vodafone ono,s....",2


In [61]:
utes_found = [ute for el in filtered.utes.values.tolist() for ute in el]
utes_matching = (len(list(set(utes_found))) / len(df_utes)) * 100
print(utes_matching)

56.61750622486114


In [64]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                      'display.max_colwidth',None
                       ):
    display(filtered.head(30))

,NIF,FullName,Name,Province,NIFtype,CompanyType,CompanyDescription,id_tender,utes,utes_length
637,01178902z,rosa granados,rosa granados,None,DNI,None,None,"[https://contrataciondelestado.es/sindicacion/datosabiertosmenores/3043580, https://contrataciondelestado.es/sindicacion/licitacionesperfilcontratante/6103439, https://contrataciondelestado.es/sindicacion/licitacionesperfilcontratante/8137452, https://contrataciondelestado.es/sindicacion/licitacionesperfilcontratante/11174681, https://contrataciondelestado.es/sindicacion/licitacionesperfilcontratante/5897380, https://contrataciondelestado.es/sindicacion/datosabiertosmenores/3397829]",[gesnaer consulting s.l.n.e. rosa granados u.t.e. ley 18 1982],1
943,01939568r,alejandro,alejandro,None,DNI,None,None,[https://contrataciondelestado.es/sindicacion/datosabiertosmenores/7059088],"[u.t.e. antonio lópez sánchez,alejandro martínez ferrer,enrique andrés casany y rafael pérez gamón ley 18 1982 u.t.e. ies azorín petrer ., balam consultotes s.l. alejandro martinez del rio u.t.e. ley 18 1982, alejandro gallego urbano david tapias monné gerard puig freixas jordi adell roig u.t.e, xoaquín monteagudo romero alejandro martín lópez, antonio lópez sánchez,alejandro martínez ferrer,enrique andrés casany,ignacio gazo lahuerta y rafael pérez gamón u.t.e. ley 18 1982]",5
1213,02459022t,andres perea ortega,andres perea ortega,None,DNI,None,None,[https://contrataciondelestado.es/sindicacion/plataformasagregadassinmenores/10428747],[euroestudios s.l. andres perea ortega u.t.e.],1
1229,02497147z,alicia torres gonzález,alicia torres gonzález,None,DNI,None,None,"[https://contrataciondelestado.es/sindicacion/datosabiertosmenores/3104794, https://contrataciondelestado.es/sindicacion/datosabiertosmenores/8564342, https://contrataciondelestado.es/sindicacion/datosabiertosmenores/9805782]",[u.t.e. alicia torres gonzález y artec 4 s.l.p.],1
8493,07016148k,construcciones perez,construcciones perez,None,DNI,None,None,"[https://contrataciondelestado.es/sindicacion/datosabiertosmenores/6275777, https://contrataciondelestado.es/sindicacion/datosabiertosmenores/7036837, https://contrataciondelestado.es/sindicacion/datosabiertosmenores/7423939, https://contrataciondelestado.es/sindicacion/datosabiertosmenores/8300222, https://contrataciondelestado.es/sindicacion/datosabiertosmenores/8362663, https://contrataciondelestado.es/sindicacion/datosabiertosmenores/8593336, https://contrataciondelestado.es/sindicacion/datosabiertosmenores/8906311, https://contrataciondelestado.es/sindicacion/datosabiertosmenores/9588037, https://contrataciondelestado.es/sindicacion/datosabiertosmenores/9875525, https://contrataciondelestado.es/sindicacion/datosabiertosmenores/10880984, https://contrataciondelestado.es/sindicacion/datosabiertosmenores/11488110]","[u.t.e. construcciones perez jimenez s.l. albaida infraestructuras s.a., construcciones perez jimenez,s.l.y albaida infraestructuras,s.a.ute ley 18]",2
8765,07270702b,manuel jesus piriz gil,manuel jesus piriz gil,None,DNI,None,None,[https://contrataciondelestado.es/sindicacion/licitacionesperfilcontratante/10059149],[manuel jesus piriz gil - maria navarro cifuentes u.t.e.],1
12498,09331668q,joaquín,joaquín,None,DNI,None,None,[https://contrataciondelestado.es/sindicacion/datosabiertosmenores/8524007],"[josep maria mezquida casases joaquín solé mir u.t.e, jeremías sampedro rodríguez - joaquín lópez - fando de castro u.t.e.]",2
14202,11111111h,agencia tributaria,agencia tributaria,None,DNI,None,None,"[https://contrataciondelestado.es/sindicacion/datosabiertosmenores/9404741, https://contrataciondelestado.es/sindicacion/datosabiertosmenores/9404740, https://contrataciondelestado.es/sindicacion/datosabiertosmenores/9404725, https://contrataciondelestado.es/sindicacion/datosabiertosmenores/9404724, https://contrataciondelestado.es/sindicacion/datosabiertosmenores/9404722, https://contrataciondelestado.es/sindicacion/datosabiertosmenores/10093038, https://contrataciondelestado.es/sindicaci